# 强化学习
## 一、介绍

强化学习包括智能体(Agent)和环境(environment)两大对象，在一个离散时间序列$t=0,1,2,...$中，智能体在每一个时刻$t$从环境中观测到状态$s_t$,并通过动作$a_t$与环境交互，环境会转移到新的状态$s_{t+1}$，同时给出一个立即奖励(reward)$r_{t+1}$被智能体观测。如此循环下去，智能体会得到**观测序列**$s_0, a_0, r_1, s_1, a_1, r_2, ...$。

把每一时刻的回报用随机变量$R_t$表示，t时刻后的累计奖励为

$$G_t = R_{t+1} + \gamma R_{t+2} + ... = \sum_{k=1}^\infty \gamma^k R_{t+k+1}$$

$\gamma \in [0, 1) \ $代表折扣因子，强化学习要求智能体在环境中交互，利用交互得到的观测序列找到最优的**策略**(状态到行为的映射)使累积期望奖励$\mathbb{E}[G_t]$最大化。和监督学习相比，强化学习有如下三个特点 1. 没有显式的带标签的数据 2. 有延迟的惩罚或奖励 3. 获得数据的分布与智能体行动(Actions)的分布有关。

强化学习的trade-off: 
1. 探索(exploration)，包括探索环境未知的状态，未知的行动等，来获得更多的关于奖励(reward)的信息。
2. 挖掘(exploitation)，即对已知的信息进行优化，来最大化奖励。


## 二、基本概念
### 2.1 马尔可夫决策过程
马尔可夫决策过程(Markov decision process, MDP)是一个五元组$<S, A, P, R, \gamma>$
* 状态空间 $S$，即环境可能处在所有状态的集合。更准确地说，状态指智能体(Agent)观察到环境的状态(默认环境的状态可以被完全观测)
* 行为空间 $A$，即智能体可能采取的行动的集合
* 状态转移概率 $P_{ss'}^a = P(S_{t+1} = s'|S_t = s, A_t = a)$，即给定前一时刻环境状态$s$和智能体行为$a$的情况下，环境状态在当前时刻变为$s'$的概率
* 奖励函数 $R_s ^a = \mathbb{E}[R_{t+1}|S_t = s, A_t = a]$，即给定前一时刻环境状态$s$和智能体行为$a$的情况下，环境在当前时刻给智能体奖励的期望
* 折扣系数 $\gamma \in [0, 1)$ 

MDP具有Markov性，即给定当前时刻状态和行为的情况下，下一时刻的状态与之前的状态和行为条件独立。
$$S_{t+1} \perp S_{t-1}, A_{t-1}, ..., S_{0}, A_{0}|S_{t}, A_{t}$$



### 2.2 策略、值函数与模型
智能体要完成强化学习任务，可以通过策略，值函数和模型三个中的一个或多个实现

#### 2.2.1 策略
策略(policy)有两种定义方式
1. 确定性策略定义，从状态到行为的映射$ \ \pi: S\rightarrow A$。
2. 随机性策略定义，即$\pi(a|s) = P(A_t=a|S_t=s)$。

注意：策略是**稳定的**(stationary)，即与时间无关，但智能体可以随着时间在线地更新策略。后面$\pi(a|s)$代表随机性策略，$\pi(s)$代表确定性策略。

给定MDP: $M = <S, A, P, R, \gamma>$和策略$\pi$，则状态序列$S_1, S_2, ...,$是一个Markov过程$<S, P^\pi>$。状态和奖励序列$S_1, R_1, S_2, R_2, ...$是一个Markov奖励过程$<S, P^{\pi}, R^{\pi}, \gamma>$，满足如下等式

$$P_{ss'}^{\pi} = \sum_{a\in A} \pi(a|s)P_{ss'}^a$$


#### 2.2.2 值函数
值函数分为状态值函数和状态-行为值函数(简称行为值函数)，用来对策略好坏进行评价，指导智能体选择动作。

* 一、 **状态值函数**

$$V_{\pi}(s) = \mathbb{E}[G_t|S_t=s] = \mathbb{E}[R_{t+1} + \gamma R_{t+2} + ...|S_t = s]$$

表示从状态s开始，执行当前策略$\pi$获得的期望回报，用来衡量智能体所处状态s的价值大小。

* 二、 **状态-行为值函数**

$$Q_{\pi}(s, a) = \mathbb{E}[G_t|S_t = s, A_t = a] = \mathbb{E}[R_{t+1} + \gamma R_{t+2} + ...|S_t = s, A_t = a]$$

表示给定策略$\pi$时，根据当前状态s执行某一具体行为a所获得的期望回报。

#### 2.2.3 模型
在强化学习任务中，智能体不知道系统的状态转移概率$P_{ss'}^a$和奖励函数$R_s ^a$，但可以通过与环境交互、采样对其进行建模，模拟环境的运行机制。一般说模型已知，即智能体估计了状态转移概率$P_{ss'}^a$和奖励函数$R_s ^a$，它是环境实际运行机制的近似。

根据智能体在解决强化学习问题时，是否建立环境动力学模型，可以将其分为 1. 有模型的方法，比如动态规划 2. 无模型方法，比如蒙特卡洛，时序差分等等。


### 2.3 Bellman方程
Bellman方程是一组方程，用来描述状态值函数、行为值函数之间存在的关系。比如

$$\begin{aligned}
V_{\pi}(s) &= \mathbb{E}[R_{t+1} + \gamma R_{t+2} + ...|S_t = s] \\
        &= \mathbb{E}[R_{t+1}|S_t = s] + \gamma \mathbb{E}[R_{t+2} + \gamma R_{t+3} + ...|S_t = s] \\
        &= \sum_{a\in A} \pi(a|s) R_s^a + \gamma \sum_{a\in A} \sum_{s'\in S} V_{\pi}(s') P_{ss'}^a \pi(a|s)
\end{aligned}$$

令$V = V_{\pi}(s)$为列向量矩阵;$R = \sum_{a\in A} \pi(a|s) R_s^a$为列向量矩阵；$P = \sum_{a\in A} P_{ss'}^a \pi(a|s)$为转移概率矩阵，满足每行元素非负且和为1。Bellman方程可以重写为：

$$V = R + \gamma PV$$

**定理**：上式Bellman方程有唯一解$V^* = (I - \gamma P)^{-1}R$

**证明**：由于P是状态转移矩阵，元素非负，且行和为1，则

$$||P||_\infty = 1$$

由于$\gamma \in [0, 1)$

$$||\gamma P||_\infty < 1$$

由于矩阵的谱半径不大于矩阵的任意范数，则

$$\rho(\gamma P) = \max_{i}\{|\lambda_i|\} \leq ||\gamma P||_\infty < 1 $$

说明矩阵$\gamma P$特征值绝对值严格小于1，则$I - \gamma P$可逆，有唯一解。换句话说，**已知系统转移概率、奖励函数和策略的情况下，状态价值函数被唯一确定**。



### 2.4 最优策略和最优值函数
策略$\pi^*$是最优策略，当且仅当对于所有状态$s\in S$状态价值函数最大。

$$\forall s \in S, V_{\pi^*}(s)=\max_{\pi}V_{\pi}(s)$$

任何MDP存在一个最优策略，最优策略$\pi^*$对应的状态价值函数称为**最优状态值函数**，对应的行为价值函数称为**最优行为值函数**。

$$\forall s \in S, Q_{\pi^*}(s, a)=\max_{\pi}Q_{\pi}(s, a)$$


由行为值函数和状态值函数之间的Bellman方程，可得两者之间的关系

$$Q_{\pi^*}(s, a) = R_s^a + \gamma \sum_{s' \in S}P_{ss'}^a V_{\pi^*}(s')$$

最优状态值函数和最优行为值函数之间还存在如下关系

$$\forall s \in S, V_{\pi^*}(s) = \max_{\pi}\sum_{a\in A}\pi(a|s)Q_{\pi^*}(s, a)$$

证明：

$$V_{\pi^*}(s) = \sum_{a\in A}\pi^*(a|s)Q_{\pi^*}(s, a) \leq \max_{\pi}\sum_{a\in A}\pi(a|s)Q_{\pi^*}(s, a)$$

而如果对$\forall s \in S$有小于号成立，则意味着最大化操作找到了更优的策略。

特别的，当策略是确定性策略时

$$\forall s \in S, V_{\pi^*}(s) = \max_{a\in A} Q_{\pi^*}(s, a)$$
